# First Processing Data

In [ ]:
! wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz

--2020-11-16 12:49:35--  http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.162
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.162|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz [following]
--2020-11-16 12:49:35--  https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4996278331 (4.7G) [application/x-gzip]
Saving to: ‘food-101.tar.gz’

food-101.tar.gz     100%[===================>]   4.65G  10.4MB/s    in 8m 1s   

2020-11-16 12:57:36 (9.91 MB/s) - ‘food-101.tar.gz’ saved [4996278331/4996278331]



In [ ]:
!tar -xzf food-101.tar.gz

In [ ]:
import os
classes = os.listdir('food-101/images')
os.makedirs('food-101/train')
for item in classes:
  os.system(f'mv food-101/images/{item} food-101/train/')

In [ ]:
import json
fil = open('food-101/meta/test.json')
test_json = json.loads(fil.readline())
fil.close()

In [ ]:
!mkdir food-101/test

In [ ]:
# os.makedirs('food-101/test')
for item in test_json.keys():
  os.makedirs(f'food-101/test/{item}')
  print(item)
  for img in test_json[item]:
    os.system(f'mv food-101/train/{img}.jpg food-101/test/{img}.jpg')

churros
hot_and_sour_soup
samosa
sashimi
pork_chop
spring_rolls
panna_cotta
beef_tartare
greek_salad
foie_gras
tacos
pad_thai
poutine
ramen
pulled_pork_sandwich
bibimbap
beignets
apple_pie
crab_cakes
risotto
paella
steak
baby_back_ribs
miso_soup
frozen_yogurt
club_sandwich
carrot_cake
falafel
bread_pudding
chicken_wings
gnocchi
caprese_salad
creme_brulee
escargots
chocolate_cake
tiramisu
spaghetti_bolognese
mussels
scallops
baklava
edamame
macaroni_and_cheese
pancakes
garlic_bread
beet_salad
onion_rings
red_velvet_cake
grilled_salmon
chicken_curry
deviled_eggs
caesar_salad
hummus
fish_and_chips
lasagna
peking_duck
guacamole
strawberry_shortcake
clam_chowder
croque_madame
french_onion_soup
beef_carpaccio
fried_rice
donuts
gyoza
ravioli
fried_calamari
spaghetti_carbonara
french_toast
lobster_bisque
ceviche
bruschetta
french_fries
shrimp_and_grits
filet_mignon
hamburger
dumplings
tuna_tartare
sushi
cheese_plate
eggs_benedict
cup_cakes
takoyaki
chocolate_mousse
breakfast_burrito
hot_dog
ma

In [ ]:
!rm -r food-101/images/
!rm -r food-101/meta/
!rm food-101/README.txt
!rm food-101/license_agreement.txt

In [ ]:
!tar -czf food-101.tar.gz food-101

In [ ]:
!cp food-101.tar.gz drive/My\ Drive/Datasets

In [ ]:
!rm -r food-101/

# MobileNet V2

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
!cp drive/My\ Drive/Datasets/food-101.tar.gz ./

In [ ]:
!tar -xzf food-101.tar.gz

In [ ]:
shape = (224,224,3)
num_classes=101
mobnetv2 = tf.keras.applications.MobileNetV2(include_top=False, input_shape=shape, weights='imagenet')
mobnetv2.trainable = False
mobnetv2.summary()

9412608/9406464 [==============================] - 0s 0us/step
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
________________

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D, Conv2D, MaxPooling2D, ZeroPadding2D, BatchNormalization

out_model = mobnetv2.output
out_model = GlobalAveragePooling2D()(out_model)
out_model = Dense(1024,activation='relu')(out_model)
out_model = Dense(512,activation='relu')(out_model)
out_model = Dense(num_classes,activation='softmax')(out_model)

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

model = Model(inputs=mobnetv2.input, outputs=out_model)

model.compile(optimizer=Adam(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_gen = ImageDataGenerator(rotation_range=20, 
                               rescale=1./255,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               shear_range=0.1,
                               zoom_range=0.1,
                               horizontal_flip=True,
                               fill_mode='nearest')

test_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_path = './food-101/train'
test_path = './food-101/test'

batch_size = 64
shape = (224,224,3)

train_data = train_gen.flow_from_directory(
    train_path, 
    target_size=shape[:-1], 
    batch_size=batch_size,
    )

test_data = test_gen.flow_from_directory(
    test_path, 
    target_size=shape[:-1], 
    batch_size=batch_size,
    shuffle=False,
)

Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

earlyStopping = EarlyStopping(
    monitor='val_loss',
    min_delta = 0, 
    patience = 3,
    verbose = 1,
    restore_best_weights = True
)

checkpoint = ModelCheckpoint(
    './Models/seefood.h5',
    monitor="val_loss",
    mode="min",
    save_best_only = True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.2,
    patience = 2,
    verbose = 1,
    min_delta = 0.0001
)

callbacks = [earlyStopping, checkpoint, reduce_lr]

In [ ]:
n_train = 75750
n_test = 25250
epochs = 32
history = model.fit(
    train_data,
    steps_per_epoch = n_train // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = test_data,
    validation_steps = n_test // batch_size    
)

Epoch 1/20
1183/1183 [==============================] - ETA: 0s - loss: 3.7314 - accuracy: 0.1247
Epoch 00001: val_loss improved from inf to 3.21846, saving model to ./Models/seefood.h5
1183/1183 [==============================] - 4515s 4s/step - loss: 3.7314 - accuracy: 0.1247 - val_loss: 3.2185 - val_accuracy: 0.1711
Epoch 2/20
  26/1183 [..............................] - ETA: 56:57 - loss: 3.3761 - accuracy: 0.1581

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-47ab253366a5>", line 10, in <module>
    validation_steps = n_test // batch_size
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py", line 108, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py", line 1098, in fit
    tmp_logs = train_function(iterator)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/eager/def_function.py", line 780, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/eager/def_function.py", line 807, in _call
    return self._stateless_fn(*args, **kwds)  # pylint: disable=not-callable
  File "/usr/local/lib/python3.6/dist-packages/tenso

KeyboardInterrupt: ignored

In [ ]:
! cp -r ./Models ./drive/My\ Drive/

In [ ]:
import mail_file
title= 'SeeFood',
path= 'My Drive/Models/seefood.h5',
acc= 80,
val_acc= 82,
loss= 0.0500,
val_loss= 0.1230

mail_file.send_mail(title, path, acc, val_acc, loss, val_loss)

# Inception Net V3

In [3]:
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [4]:
!cp drive/My\ Drive/Datasets/food-101.tar.gz ./

In [5]:
!tar -xzf food-101.tar.gz

In [6]:
shape = (299,299,3)
num_classes=101
inceptionv3 = tf.keras.applications.InceptionV3(include_top=False, input_shape=shape, weights='imagenet')
inceptionv3.trainable = False
inceptionv3.summary()

In [ ]:
from tensorflow.keras.layers import AveragePooling2D, Flatten, Dropout, Dense
output_model = inceptionv3.output
output_model = AveragePooling2D((8,8))(output_model)
output_model = Dropout(0.4)(output_model)
output_model = Flatten()(output_model)
output_model = Dense(num_classes, kernel_initializer='glorot_uniform', activation='softmax')(output_model)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

model = Model(inputs=inceptionv3.input, outputs=output_model)
model.compile(optimizer=Adam(learning_rate=0.005), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_gen = ImageDataGenerator(rotation_range=20, 
                               rescale=1./255,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               shear_range=0.1,
                               zoom_range=0.1,
                               horizontal_flip=True,
                               fill_mode='nearest')

test_gen = ImageDataGenerator(rescale=1./255)

In [8]:
train_path = './food-101/train'
test_path = './food-101/test'

batch_size = 64

train_data = train_gen.flow_from_directory(
    train_path, 
    target_size=shape[:-1], 
    batch_size=batch_size,
    )

test_data = test_gen.flow_from_directory(
    test_path, 
    target_size=shape[:-1], 
    batch_size=batch_size,
    shuffle=False,
)

Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.


In [13]:
import json
mapping = json.dumps(test_data.class_indices)
fil = open('class_mapping.json', mode='w', )
fil.writelines(mapping)
fil.close()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau

os.makedirs('./Models/')
earlyStopping = EarlyStopping(
    monitor='val_loss',
    min_delta = 0, 
    patience = 3,
    verbose = 1,
    restore_best_weights = True
)

checkpoint = ModelCheckpoint(
    './Models/seefood.h5',
    monitor="val_loss",
    mode="min",
    save_best_only = True,
    verbose=1
)

# def schedule(epoch):
#     if epoch < 15:
#         return .005
#     elif epoch < 28:
#         return .001
#     else:
#         return .0005

# lr_scheduler = LearningRateScheduler(schedule)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 2, verbose = 1, min_delta = 0.0001)

callbacks = [earlyStopping, checkpoint, reduce_lr]

In [ ]:
os.makedirs('./Models/')

In [ ]:
n_train = 75750
n_test = 25250
epochs = 40
history = model.fit(
    train_data,
    steps_per_epoch = n_train // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = test_data,
    validation_steps = n_test // batch_size    
)

Epoch 1/40
1183/1183 [==============================] - ETA: 0s - loss: 3.1163 - accuracy: 0.3708
Epoch 00001: val_loss improved from inf to 2.13394, saving model to ./Models/seefood.h5

Epoch 00001: val_loss improved from inf to 2.13394, saving model to ./Models/seefood.h5
1183/1183 [==============================] - 1962s 2s/step - loss: 3.1163 - accuracy: 0.3708 - val_loss: 2.1339 - val_accuracy: 0.5250
Epoch 2/40
Epoch 2/40
1183/1183 [==============================] - ETA: 0s - loss: 3.1754 - accuracy: 0.4222
Epoch 00002: val_loss improved from 2.13394 to 2.02818, saving model to ./Models/seefood.h5

Epoch 00002: val_loss improved from 2.13394 to 2.02818, saving model to ./Models/seefood.h5
1183/1183 [==============================] - 1972s 2s/step - loss: 3.1754 - accuracy: 0.4222 - val_loss: 2.0282 - val_accuracy: 0.5596
Epoch 3/40
Epoch 3/40
1183/1183 [==============================] - ETA: 0s - loss: 3.3254 - accuracy: 0.4288
Epoch 00003: val_loss did not improve from 2.02818
1